# RGU-IIT CBR Test Embedding Generation

## Download Libraries

In [ ]:
!pip install -U angle-emb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.3/195.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━

In [ ]:
!pip install datasets peft -q

In [ ]:
!pip install mistralai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

api_key = "XXX"

client = MistralClient(api_key=api_key)

## Import Libraries

In [ ]:
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModel
import torch

In [ ]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

In [ ]:
import torch

In [ ]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Load Dataset

In [ ]:
df_mistral = pd.read_csv("resources/generated_results.csv")

In [ ]:
df_mistral.head()

,case_index,no_rag_pipeline_result,normal_bert_hybrid_snippet_k1_indexes,normal_bert_hybrid_snippet_k1_context,normal_bert_hybrid_snippet_k1_result,normal_bert_hybrid_case_k1_indexes,normal_bert_hybrid_case_k1_context,normal_bert_hybrid_case_k1_result,legal_bert_hybrid_snippet_k1_indexes,legal_bert_hybrid_snippet_k1_context,...,question_angle_bert_matching_embeddings,question_normal_bert_retrieval_embeddings,question_legal_bert_retrieval_embeddings,question_angle_bert_retrieval_embeddings,answer_normal_bert_matching_embeddings,answer_legal_bert_matching_embeddings,answer_angle_bert_matching_embeddings,answer_normal_bert_retrieval_embeddings,answer_legal_bert_retrieval_embeddings,answer_angle_bert_retrieval_embeddings
0,0,In the case of Econ Holdings Pty Ltd v Sims Lo...,[1166],"""13. It can therefore be said in this case tha...",In the case of Econ Holdings Pty Ltd v Sims Lo...,[1166],"""Question: ""In the case of Garawin Pty Ltd v 1...",In the case Econ Holdings Pty Ltd v Sims Lockw...,[106],"""light of authorities commencing at least from...",...,"[-0.3710246682167053, -0.30695846676826477, -0...","[0.09509215503931046, -0.2021857500076294, -0....","[-0.0023545799776911736, -0.10421659797430038,...","[-0.4049617350101471, -0.05314958840608597, -0...","[-0.22807049751281738, -0.09595131129026413, 0...","[-0.11683186143636703, -0.03316358104348183, 0...","[-0.4248591661453247, -0.04108070954680443, -0...","[-0.22192685306072235, -0.09357594698667526, 0...","[-0.09864497184753418, -0.024802930653095245, ...","[-0.4455259144306183, 0.04232563078403473, -0...."
1,1,The decision to disclose specific details in E...,[347],""" * In Event 38832023 - the date of birth and ...",According to the given context and the New Sou...,[347],"""Question: ""What type of information was redac...",Based on the case of Robinson v Commissioner o...,[1430],"""45The risk must also be linked ""to the safety...",...,"[0.624129056930542, -0.3914183974266052, -0.04...","[0.02651604823768139, -0.013784815557301044, -...","[-0.05422603711485863, 0.1209280863404274, -0....","[0.36398348212242126, -0.16158992052078247, -0...","[-0.24886399507522583, 0.24751828610897064, -0...","[0.04563914239406586, 0.19787001609802246, -0....","[0.5655755996704102, -0.0976831465959549, -0.1...","[-0.2162674218416214, 0.2396380603313446, -0.0...","[0.05516544729471207, 0.20160458981990814, -0....","[0.5197416543960571, -0.056952767074108124, -0..."
2,2,"Case 1: In the first case, Slattery J heard a ...",[1145],""" 19 However, it is not just the statement of ...",In the first case heard before Slattery J in t...,[1145],"""Question: ""In the case of Neeson v Amora Comp...",In the case of Neeson v Amora Company Limited ...,[488],""" However, the words cannot be taken in isolat...",...,"[-0.322487473487854, -0.10779520124197006, -0....","[0.06910744309425354, -0.13112026453018188, -0...","[-0.0018813080387189984, 0.2632799446582794, 0...","[-0.3880188763141632, 0.0675685852766037, -0.7...","[-0.29932308197021484, -0.08719420433044434, 0...","[-0.1529170125722885, 0.12348916381597519, 0.1...","[-0.3596920967102051, -0.22549670934677124, -0...","[-0.29801926016807556, -0.09234002232551575, 0...","[-0.14582818746566772, 0.14794474840164185, 0....","[-0.379778653383255, -0.15069353580474854, -0...."
3,3,Rule 8.21(1)(d) of the Federal Court Rules per...,[563],"""116 In short, as Balkin and Davis explain in ...","In the context of the Federal Court Rules, an ...",[563],"""Question: ""What is the fundamental issue when...","In the context of Federal Court Rules, an amen...",[1881],"""37 Consistent with this reasoning, Gordon J f...",...,"[-0.08438273519277573, -0.07412683963775635, -...","[0.0569213330745697, -0.042861513793468475, -0...","[-0.09847217798233032, 0.03212566673755646, -0...","[-0.20830093324184418, 0.040662266314029694, -...","[-0.2801944613456726, 0.016026843339204788, -0...","[-0.213663250207901, 0.0026465130504220724, -0...","[0.09185919910669327, 0.03754888102412224, 

## Embedding Assets

In [ ]:
legal_tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
legal_model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

normal_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
normal_model = BertModel.from_pretrained("bert-base-uncased")

angle_model_id = 'SeanLee97/angle-bert-base-uncased-nli-en-v1'
angle_tokenizer = AutoTokenizer.from_pretrained(angle_model_id)
angle_model = AutoModel.from_pretrained(angle_model_id).cuda()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/708 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
def get_legal_bert_embeddings(sentence, prompt):
    text = prompt.format(text=sentence)
    inputs = legal_tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = legal_model(**inputs)

    last_hidden_states = outputs.last_hidden_state
    sentence_embedding = torch.mean(last_hidden_states, dim=1)

    return sentence_embedding[0].tolist()

In [ ]:
def get_normal_bert_embeddings(sentence, prompt):
    text = prompt.format(text=sentence)
    input_ids = normal_tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=512)
    input_ids = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        outputs = normal_model(input_ids)
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()  # Take mean of token embeddings

    return embeddings.tolist()

In [ ]:
def get_angle_bert_embeddings(sentence, prompt):
    text = prompt.format(text=sentence)
    input_ids = angle_tokenizer([text], return_tensors='pt', truncation=True, max_length=512)

    for k, v in input_ids.items():
      input_ids[k] = v.cuda()

    hidden_state = angle_model(**input_ids).last_hidden_state
    vec = (hidden_state[:, 0] + torch.mean(hidden_state, dim=1)) / 2.0

    return vec[0].tolist()

In [ ]:
def get_mistral_embeddings(sentence, prompt):
  text = prompt.format(text=sentence)
  embeddings_response = client.embeddings(
        model="mistral-embed",
        input=[text],
    )

  return embeddings_response.data[0].embedding

In [ ]:
# Test Embeddings:
legal_sentence = "The court dismisses the case due to lack of evidence."
normal_sentence = "This is a normal sentence for BERT embeddings."

empty_prompt = "{text}"

legal_embeddings = get_legal_bert_embeddings(normal_sentence, empty_prompt)
normal_embeddings = get_normal_bert_embeddings(normal_sentence, empty_prompt)
angle_embeddings = get_angle_bert_embeddings(normal_sentence, empty_prompt)
mistral_embeddings = get_mistral_embeddings(normal_sentence, empty_prompt)

print("Legal BERT Embeddings:", len(legal_embeddings))
print("Normal BERT Embeddings:", len(normal_embeddings))
print("Angle BERT Embeddings:", len(angle_embeddings))
print("Mistral Embeddings:", len(mistral_embeddings))

Legal BERT Embeddings: 768
Normal BERT Embeddings: 768
Angle BERT Embeddings: 768
Mistral Embeddings: 1024


In [ ]:
PROMPT_DICT = {
    "Matching": "{text}",
    "Retrieval_A": 'Summarize sentence "{text}" in one word:',
    "Retrieval_B": 'You can only output one word. Summarize "{text}":"',
    "Retrieval_C": 'Represent this sentence for searching relevant passages: {text}',
}

In [ ]:
PROMPT_DICT["Retrieval_C"]

'Represent this sentence for searching relevant passages: {text}'

## Get Embeddings

In [ ]:
!mkdir assets

## Generate Embeddings

In [ ]:
new_df_mistral = pd.DataFrame(columns=df_mistral.columns)

In [ ]:
df_rows = df_mistral.copy()

In [ ]:
i = 0
for index, row in tqdm(list(df_rows.iterrows())):
  i+=1
  try:
    # Get Matching and Retreival Embeddings for all results
    for result_column in result_columns:
        angle_bert_retireval_emb =  get_angle_bert_embeddings(row[result_column], PROMPT_DICT["Retrieval_C"])
        normal_bert_retireval_emb =  get_normal_bert_embeddings(row[result_column], PROMPT_DICT["Retrieval_C"])
        legal_bert_retireval_emb =  get_legal_bert_embeddings(row[result_column], PROMPT_DICT["Retrieval_C"])
        mistral_retireval_emb =  get_mistral_embeddings(row[result_column], PROMPT_DICT["Retrieval_C"])

        angl_bert_matching_emb =  get_angle_bert_embeddings(row[result_column], PROMPT_DICT["Matching"])
        normal_bert_matching_emb =  get_normal_bert_embeddings(row[result_column], PROMPT_DICT["Matching"])
        legal_bert_matching_emb =  get_legal_bert_embeddings(row[result_column], PROMPT_DICT["Matching"])
        mistral_matching_emb =  get_mistral_embeddings(row[result_column], PROMPT_DICT["Matching"])

        # Add Colum tyo row
        row[f"{result_column}_angle_bert_retireeal_emb"] = angle_bert_retireval_emb
        row[f"{result_column}_normal_bert_retireeal_emb"] = normal_bert_retireval_emb
        row[f"{result_column}_legal_bert_retireeal_emb"] = legal_bert_retireval_emb
        row[f"{result_column}_mistral_retireeal_emb"] = mistral_retireval_emb

        row[f"{result_column}_angle_bert_matching_emb"] = angl_bert_matching_emb
        row[f"{result_column}_normal_bert_matching_emb"] = normal_bert_matching_emb
        row[f"{result_column}_legal_bert_matching_emb"] = legal_bert_matching_emb
        row[f"{result_column}_mistral_matching_emb"] = mistral_matching_emb


    new_df_mistral = new_df_mistral.append(row, ignore_index=True)

  except Exception as e:
    print(e)
    print(f"Error in {i}")

100%|██████████| 35/35 [17:42<00:00, 30.36s/it]


In [ ]:
new_df_mistral.head()

,case_index,no_rag_pipeline_result,normal_bert_hybrid_snippet_k1_indexes,normal_bert_hybrid_snippet_k1_context,normal_bert_hybrid_snippet_k1_result,normal_bert_hybrid_case_k1_indexes,normal_bert_hybrid_case_k1_context,normal_bert_hybrid_case_k1_result,legal_bert_hybrid_snippet_k1_indexes,legal_bert_hybrid_snippet_k1_context,...,angle_bert_hybrid_case_k3_result_legal_bert_matching_emb,angle_bert_hybrid_case_k3_result_mistral_matching_emb,answer_angle_bert_retireeal_emb,answer_normal_bert_retireeal_emb,answer_legal_bert_retireeal_emb,answer_mistral_retireeal_emb,answer_angle_bert_matching_emb,answer_normal_bert_matching_emb,answer_legal_bert_matching_emb,answer_mistral_matching_emb
0,0,In the case of Econ Holdings Pty Ltd v Sims Lo...,[1166],"""13. It can therefore be said in this case tha...",In the case of Econ Holdings Pty Ltd v Sims Lo...,[1166],"""Question: ""In the case of Garawin Pty Ltd v 1...",In the case Econ Holdings Pty Ltd v Sims Lockw...,[106],"""light of authorities commencing at least from...",...,"[-0.23479348421096802, -0.1956131011247635, -0...","[-0.057037353515625, 0.047210693359375, 0.0238...","[-0.4455259144306183, 0.04232563078403473, -0....","[-0.22192685306072235, -0.09357594698667526, 0...","[-0.09864497184753418, -0.024802930653095245, ...","[-0.045867919921875, 0.04443359375, 0.03314208...","[-0.4248591661453247, -0.04108070954680443, -0...","[-0.22807049751281738, -0.09595131129026413, 0...","[-0.11683186143636703, -0.03316358104348183, 0...","[-0.047271728515625, 0.042999267578125, 0.0375..."
1,1,The decision to disclose specific details in E...,[347],""" * In Event 38832023 - the date of birth and ...",According to the given context and the New Sou...,[347],"""Question: ""What type of information was redac...",Based on the case of Robinson v Commissioner o...,[1430],"""45The risk must also be linked ""to the safety...",...,"[-0.08831224590539932, 0.20533089339733124, -0...","[-0.049102783203125, 0.044189453125, 0.0422058...","[0.5197416543960571, -0.056952767074108124, -0...","[-0.2162674218416214, 0.2396380603313446, -0.0...","[0.05516544729471207, 0.20160458981990814, -0....","[-0.04022216796875, 0.03497314453125, 0.040283...","[0.5655755996704102, -0.0976831465959549, -0.1...","[-0.24886399507522583, 0.24751828610897064, -0...","[0.04563914239406586, 0.19787001609802246, -0....","[-0.033843994140625, 0.0237884521484375, 0.043..."
2,2,"Case 1: In the first case, Slattery J heard a ...",[1145],""" 19 However, it is not just the statement of ...",In the first case heard before Slattery J in t...,[1145],"""Question: ""In the case of Neeson v Amora Comp...",In the case of Neeson v Amora Company Limited ...,[488],""" However, the words cannot be taken in isolat...",...,"[0.004998308140784502, 0.20312686264514923, -0...","[-0.04071044921875, 0.06158447265625, 0.043884...","[-0.379778653383255, -0.15069353580474854, -0....","[-0.29801926016807556, -0.09234002232551575, 0...","[-0.14582818746566772, 0.14794474840164185, 0....","[-0.0169219970703125, 0.049468994140625, 0.032...","[-0.3596920967102051, -0.22549670934677124, -0...","[-0.29932308197021484, -0.08719420433044434, 0...","[-0.1529170125722885, 0.12348916381597519, 0.1...","[-0.0208587646484375, 0.042205810546875, 0.033..."
3,3,Rule 8.21(1)(d) of the Federal Court Rules per...,[563],"""116 In short, as Balkin and Davis explain in ...","In the context of the Federal Court Rules, an ...",[563],"""Question: ""What is the fundamental issue when...","In the context of Federal Court Rules, an amen...",[1881],"""37 Consistent with this reasoning, Gordon J f...",...,"[-0.2204984724521637, 0.0516708642244339, -0.0...","[-0.0506591796875, 0.029449462890625, 0.035522...","[0.034823209047317505, 0.11204800009727478, -0...","[-0.2666357159614563, 0.003912794403731823, -0...","[-0.22142720222473145, 0.016492782160639763, -...","[-0.06317138671875, 0.03729248046875, 0.032501...","[0.09185919910669327, 0.03754888102412224, -0....","[-0.2801944613456726, 0.01602

In [ ]:
new_df_mistral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Columns: 165 entries, case_index to answer_mistral_matching_emb
dtypes: object(165)
memory usage: 45.2+ KB


In [ ]:
new_df_mistral.to_csv("resources/generated_results_embddings", index=False)